In [7]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [31]:
if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")

No GPU was detected. LSTMs and CNNs can be very slow without a GPU.


## Load data

In [2]:
data_url = "https://homl.info/shakespeare"

In [3]:
filepath = tf.keras.utils.get_file("shakespeare.txt", data_url)

In [4]:
with open(filepath) as text:
    shakespeare_text = text.read()

In [5]:
print(shakespeare_text[:148])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?



## Tokenize text

In [8]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)

In [9]:
tokenizer.fit_on_texts(shakespeare_text)

In [10]:
tokenizer.texts_to_sequences(["First"])

[[20, 6, 9, 8, 3]]

In [11]:
max_id = len(tokenizer.word_index)

In [12]:
dataset_size = tokenizer.document_count

In [13]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

## Prepare data

In [14]:
train_size = dataset_size * 90 // 100

In [15]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [16]:
n_steps = 100

In [17]:
window_length = n_steps + 1

In [18]:
dataset = dataset.repeat().window(window_length, shift=1, drop_remainder=True)

In [19]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [20]:
batch_size = 32

In [21]:
dataset = dataset.shuffle(10000).batch(batch_size)

In [22]:
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [23]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [24]:
dataset = dataset.prefetch(1)

In [26]:
for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

(32, 100, 39) (32, 100)


## Build and trian model

In [27]:
model = tf.keras.Sequential([
    tf.keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id], dropout=0.2, recurrent_dropout=0.2),
    tf.keras.layers.GRU(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(max_id, activation='softmax'))
])

In [28]:
model.compile(loss="sparse_categorical_crossentropy", optimizer='adam')

In [29]:
history = model.fit(dataset, epochs=20)

Epoch 1/20
   3266/Unknown - 434s 133ms/step - loss: 1.7716

KeyboardInterrupt: 